In [1]:
import tensorflow as tf

## Getting Started

In [2]:
x1 = tf.constant(5)
x2 = tf.constant(6)

In [13]:
result = tf.multiply(x1, x2)
print(result)

Tensor("Mul_2:0", shape=(), dtype=int32)


In [16]:
sess = tf.Session()
print(sess.run(result))

30


In [17]:
sess.close()

In [19]:
with tf.Session() as sess:
    output = sess.run(result)
    print(sess.run(result))

30


In [21]:
print(output)

30


## Building a Neural Net

- A neural net is 1 (sometimes 2) layers

- A Deep NN/Learning is 2+ layers

### Simple Architecture

**Feed Forward**

1) *Input >> Weight >> Hidden Layer 1 (Activation Function >>
         Weigths >> Hidden Layer 2 (Activation Function)*
         
2) **Cost/Loss Function (i.e. Cross Entropy)** - compare output to intended output

3) **Optimization Function (Optimizer)** - minimize the loss/cost (i.e. AdamOptimizer... SGD, AdaGrad)

4) **Backpropagation**

5) **Epoch** = feedfoward + backprop


**One_Hot** = means target is "1" and the remaining outcomes are "0"

In [64]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./data/data", one_hot=True)

Extracting ./data/data/train-images-idx3-ubyte.gz
Extracting ./data/data/train-labels-idx1-ubyte.gz
Extracting ./data/data/t10k-images-idx3-ubyte.gz
Extracting ./data/data/t10k-labels-idx1-ubyte.gz


In [65]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

In [66]:
n_classes = 10
batch_size = 100

In [67]:
# height x width
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

(Input Data * Weights) + Baises

In [68]:
def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.multiply(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.multiply(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.multiply(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.multiply(l3,output_layer['weights']) + output_layer['biases']

    return output

In [69]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [70]:
train_neural_network(x)

ValueError: Dimensions must be equal, but are 784 and 500 for 'Mul_10' (op: 'Mul') with input shapes: [?,784], [784,500].